In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import warnings
import gc
import time

time_start=time.time()

In [2]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
target=train['target']
#feature_answer_base=pd.read_csv('feature_answer_base.csv')
member_info=pd.read_csv('member_info.txt',sep='\s+',names=['uid','sex','key_word','num_level','hot_level','regis_type','regis_platform',
                                                          'look_freq','a','b','c','d','e','A','B','C','D','E','salt','l_topic','topic_n'])

In [3]:
#断点

In [4]:
train=train[['qid','uid','target']]
test=test[['qid','uid']]
member_info=member_info[['uid','sex','look_freq','a','b','c','d','e','A','B','C','D','E','salt']]

In [5]:
all_data1=pd.concat([train[['qid','uid']],test])

In [6]:
all_data1=pd.merge(all_data1,member_info,how='left',on='uid')

In [7]:
from tqdm import tqdm

In [8]:
from sklearn.preprocessing import LabelEncoder
class_feat =  ['uid','qid','sex', 'look_freq','A','B','C','D','E']
encoder = LabelEncoder()
for feat in tqdm(class_feat):
    encoder.fit(all_data1[feat])
    all_data1[feat] = encoder.transform(all_data1[feat])

100%|██████████| 9/9 [00:35<00:00,  2.74s/it]


In [9]:
for feat in  tqdm(['uid','qid','sex', 'look_freq', 'a', 'b', 'c', 'd', 'e', 'A', 'B','C', 'D', 'E']):
    col_name = '{}_count'.format(feat)
    all_data1[col_name] = all_data1[feat].map(all_data1[feat].value_counts().astype(int))
    all_data1.loc[all_data1[col_name] < 2, feat] = -1
    all_data1[feat] += 1
    all_data1[col_name] = all_data1[feat].map(all_data1[feat].value_counts().astype(int))
    all_data1[col_name] = (all_data1[col_name] - all_data1[col_name].min()) / (all_data1[col_name].max() - all_data1[col_name].min())

100%|██████████| 14/14 [00:34<00:00,  2.10s/it]


In [10]:
len(all_data1)

10630880

In [11]:
all_data1.drop(['a_count','b_count','c_count','d_count','e_count','E_count'],axis=1,inplace=True)

In [12]:
all_data1.drop(['uid_count','qid_count'],axis=1,inplace=True)

In [16]:
all_data1.drop(['qid','uid'],axis=1,inplace=True)

In [17]:
all_data1

,sex,look_freq,a,b,c,d,e,A,B,C,D,E,salt,sex_count,look_freq_count,A_count,B_count,C_count,D_count
0,3,5,1,2,1,1,1,1920,169,248,802,2,297,1.000000,0.934449,0.021579,0.372720,0.212061,0.062051
1,3,2,2,1,1,1,1,1092,116,253,661,1,308,1.000000,0.245729,1.000000,1.000000,0.504125,1.000000
2,3,5,2,1,1,1,1,1380,205,1,443,2,415,1.000000,0.934449,0.024575,0.111173,0.432611,0.002824
3,3,1,2,1,1,2,1,1092,116,248,802,2,305,1.000000,1.000000,1.000000,1.000000,0.212061,0.062051
4,3,2,2,1,1,1,1,511,202,178,710,2,296,1.000000,0.245729,0.006149,0.033573,0.398259,0.044851
5,2,1,2,1,1,1,1,1380,205,1,443,2,380,0.027552,1.000000,0.024575,0.111173,0.432611,0.002824
6,2,1,2,1,1,1,1,1380,205,1,443,2,380,0.027552,1.000000,0.024575,0.111173,0.432611,0.002824
7,3,2,2,1,1,1,1,1092,116,1,130,2,520,1.000000,0.245729,1.000000,1.000000,0.432611,0.002979
8,3,5,1,2,1,1,1,1487,169,253,661,2,282,1.000000,0.934449,0.031208,0.372720,0.504125,1.000000
9,2,1,2,1,1,1,1,1092,116,373,1257,2,446,0.027552,1.000000,1.000000,1.000000,0.441927,0.129312


In [18]:
all_data1.reset_index(drop=True).to_hdf('feature_member_info_new.h5', key='data')

In [ ]:
###################################################################################